In [15]:
import pandas as pd
import numpy as np
import warnings
import pickle
from math import sqrt
warnings.filterwarnings('ignore')
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.authentication import ServicePrincipalAuthentication

In [16]:
# Connect to Workspace
ws = Workspace.from_config()
#print(ws)

## Download scaler and model from workspace

In [6]:
scaler = Model(ws,'scaler').download(exist_ok=True)

In [7]:
svc_model = Model(ws,'support-vector-classifier').download(exist_ok=True)

## Load files

In [8]:
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [9]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("svc.onnx")

In [10]:
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

## inference on test data

In [11]:
test_data = np.array([34.927778, 0.24, 7.3899, 83, 16.1000, 1])

In [12]:
# Scale data
test_data = scaler.fit_transform(test_data.reshape(1, 6))

In [13]:
pred_onx = sess.run([label_name], {input_name: test_data.astype(numpy.float32)})[0]

In [14]:
pred_onx[0]

1